In [ ]:
import deepxde as dde
import matplotlib.pyplot as plt
import numpy as np
from scipy import integrate
import torch
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
print(torch.cuda.is_available())

In [ ]:
beta = 0.0921
gamma = 0.0492
tot=1

#def func(s, i,r):
#    ds_t = miu-miu*s-beta*s*i
#    di_t = -miu*i+beta*s*i-gamma*i
#    dr_t = gamma*i-miu*r
#    return ds_t,di_t,dr_t

In [ ]:
def boundary(_, on_initial):
    return on_initial

In [ ]:
def ode_system(x, y):
    S, I,R = y[:, 0:1], y[:, 1:2], y[:,2:3]
    ds_x = dde.grad.jacobian(y, x, i=0)
    di_x = dde.grad.jacobian(y, x, i=1)
    dr_x = dde.grad.jacobian(y, x, i=2)
    return [ds_x+beta*I*S/tot, di_x-beta*I*S/tot+gamma*I, dr_x-gamma*I,S+I+R-tot]

In [ ]:
geom = dde.geometry.TimeDomain(0, 150)
ic1 = dde.icbc.IC(geom, lambda x: 0.968, boundary, component=0)
ic2 = dde.icbc.IC(geom, lambda x: 0.031667, boundary, component=1)
ic3 = dde.icbc.IC(geom, lambda x: 0.000333, boundary, component=2)
data = dde.data.PDE(geom, ode_system, [ic1,ic2,ic3], 3000, 100, num_test=100)

layer_size = [1,20,80,256,40,3]
activation = "elu"
initializer = "Glorot normal"
net = dde.nn.FNN(layer_size, activation, initializer)

In [ ]:
model = dde.Model(data, net)
model.compile("adam", lr=0.001)
losshistory, train_state = model.train(iterations=50000)
# Most backends except jax can have a second fine tuning of the solution


In [ ]:
from deepxde.utils.external import *

In [ ]:
def pack_data(train_state):
    def merge_values(values):
        if values is None:
            return None
        return np.hstack(values) if isinstance(values, (list, tuple)) else values

    y_train = merge_values(train_state.y_train)
    y_test = merge_values(train_state.y_test)
    best_y = merge_values(train_state.best_y)
    best_ystd = merge_values(train_state.best_ystd)
    return y_train, y_test, best_y, best_ystd
s = pack_data(train_state)

In [ ]:
def myplot_best_state(train_state):
    """Plot the best result of the smallest training loss.

    This function only works for 1D and 2D problems. For other problems and to better
    customize the figure, use ``save_best_state()``.

    Note:
        You need to call ``plt.show()`` to show the figure.

    Args:
        train_state: ``TrainState`` instance. The second variable returned from
            ``Model.train()``.
    """
    if isinstance(train_state.X_train, (list, tuple)):
        print(
            "Error: The network has multiple inputs, and plotting such result han't been implemented."
        )
        return

    y_train, y_test, best_y, best_ystd = pack_data(train_state)
    y_dim = best_y.shape[1]

    # Regression plot
    # 1D
    if train_state.X_test.shape[1] == 1:
        idx = np.argsort(train_state.X_test[:, 0])
        X = train_state.X_test[idx, 0]
        plt.figure()
        bestval=-1,mxval=-1
        for i in range(y_dim):
            if y_train is not None:
                plt.plot(train_state.X_train[:, 0], y_train[:, i], "ok", label="Train")
            if y_test is not None:
                plt.plot(X, y_test[idx, i], "-k", label="True")
            if i==0:
                plt.plot(X, best_y[idx, i], color='red', label="S")
            elif i==1:
                plt.plot(X, best_y[idx, i], color='green', label="I")
                if best_y[idx, i]>mxval:
                    mxval=best_y[idx, i]
                    bestval=i
            else:
                plt.plot(X, best_y[idx, i], color='blue', label="R")
            if best_ystd is not None:
                plt.plot(
                    X, best_y[idx, i] + 2 * best_ystd[idx, i], "-b", label="95% CI"
                )
                plt.plot(X, best_y[idx, i] - 2 * best_ystd[idx, i], "-b")
        plt.xlabel("x")
        plt.ylabel("y")
        plt.legend()
        print(bestval)
    # 2D
    elif train_state.X_test.shape[1] == 2:
        for i in range(y_dim):
            plt.figure()
            ax = plt.axes(projection=Axes3D.name)
            ax.plot3D(
                train_state.X_test[:, 0],
                train_state.X_test[:, 1],
                best_y[:, i],
                ".",
            )
            ax.set_xlabel("$x_1$")
            ax.set_ylabel("$x_2$")
            ax.set_zlabel("$y_{}$".format(i + 1))


In [ ]:
myplot_best_state(train_state)